In [1]:
import sys
sys.path.append("../src/")
import pandas as pd
import numpy as np
import tqdm

import ray.rllib.agents.ppo as ppo
from ray import tune
from ray.rllib.agents.ppo import PPOTrainer
import ray
from ray.tune.registry import register_env
from ray.tune.logger import pretty_print

import tensorflow as tf
import numpy as np

from ray.rllib.algorithms import Algorithm
from ray.rllib.models.preprocessors import get_preprocessor

/anaconda/envs/oppe4rl_22/lib/python3.9/site-packages/flatbuffers/compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/anaconda/envs/oppe4rl_22/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from gym_waf.envs.waf_brain_env import WafBrainEnv

In [3]:
df = pd.read_csv('../src/outputs_ppo/run_history_rllib.csv', sep=';', index_col=0)
s2 = '../src/outputs_ppo/emb_states_rllib.npy'
np_emb_state_ = np.load(s2, allow_pickle=True)
print(np_emb_state_.shape[0])
print(df.shape[0])

109
109


In [4]:
df.head()

,episode,step,original_payload,state,action,next_state,reward,win
0,0,0,;waitfor delay '0:0:__TIME__'--,;waitfor delay '0:0:__TIME__'--,25,;waitfor delay '0:0:__TIME__'--,-0.516129,0
1,0,1,;waitfor delay '0:0:__TIME__'--,;waitfor delay '0:0:__TIME__'--,19,;waitfor delay '0:0:__TIME__'--,-0.516129,0
2,0,2,;waitfor delay '0:0:__TIME__'--,;waitfor delay '0:0:__TIME__'--,14,;waitfor#~fx8;?B\ndelay#~fx8;?B\n'0:0:__TIME__'--,-0.148936,0
3,0,3,;waitfor delay '0:0:__TIME__'--,;waitfor#~fx8;?B\ndelay#~fx8;?B\n'0:0:__TIME__'--,15,;waitfor#~fx8;?B\ndelay#~fx8;?B\n'0:0:__TIME__'--,-0.148936,0
4,0,4,;waitfor delay '0:0:__TIME__'--,;waitfor#~fx8;?B\ndelay#~fx8;?B\n'0:0:__TIME__'--,11,;waitfor#~fx8;?B\ndelay#~fx8;?B\n'0:0:__TIME__'--,-0.148936,0


In [5]:
# we need to estimate the discounted reward from the current state, action until the end of the episode
episodes = list(df.episode.unique())
print(len(episodes))

20


In [6]:
print(episodes)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


In [7]:
# calculating expected reward for episode

discount = 0.99 #based in the PPO original paper, default discount
num_eps = 0
for ep in episodes:
    print(ep)
    df_ = df[df['episode'] == ep].copy()
    df_.sort_values(by=['step'], ascending=False, inplace=True)
    cum_reward = 0.0
    j = 0
    for i,step in df_.iterrows():
        if j == 0:
            cum_reward = step.reward
        else:
            cum_reward = cum_reward + discount * step.reward
        df.at[i,'exp_reward'] = cum_reward
        j += 1

In [8]:
df.head()

,episode,step,original_payload,state,action,next_state,reward,win,exp_reward
0,0,0,;waitfor delay '0:0:__TIME__'--,;waitfor delay '0:0:__TIME__'--,25,;waitfor delay '0:0:__TIME__'--,-0.516129,0,-3.605828
1,0,1,;waitfor delay '0:0:__TIME__'--,;waitfor delay '0:0:__TIME__'--,19,;waitfor delay '0:0:__TIME__'--,-0.516129,0,-3.094860
2,0,2,;waitfor delay '0:0:__TIME__'--,;waitfor delay '0:0:__TIME__'--,14,;waitfor#~fx8;?B\ndelay#~fx8;?B\n'0:0:__TIME__'--,-0.148936,0,-2.583892
3,0,3,;waitfor delay '0:0:__TIME__'--,;waitfor#~fx8;?B\ndelay#~fx8;?B\n'0:0:__TIME__'--,15,;waitfor#~fx8;?B\ndelay#~fx8;?B\n'0:0:__TIME__'--,-0.148936,0,-2.436445
4,0,4,;waitfor delay '0:0:__TIME__'--,;waitfor#~fx8;?B\ndelay#~fx8;?B\n'0:0:__TIME__'--,11,;waitfor#~fx8;?B\ndelay#~fx8;?B\n'0:0:__TIME__'--,-0.148936,0,-2.288998


In [10]:
df[df['episode'] == 0]

,episode,step,original_payload,state,action,next_state,reward,win,exp_reward
0,0,0,;waitfor delay '0:0:__TIME__'--,;waitfor delay '0:0:__TIME__'--,25,;waitfor delay '0:0:__TIME__'--,-0.516129,0,-3.605828
1,0,1,;waitfor delay '0:0:__TIME__'--,;waitfor delay '0:0:__TIME__'--,19,;waitfor delay '0:0:__TIME__'--,-0.516129,0,-3.094860
2,0,2,;waitfor delay '0:0:__TIME__'--,;waitfor delay '0:0:__TIME__'--,14,;waitfor#~fx8;?B\ndelay#~fx8;?B\n'0:0:__TIME__'--,-0.148936,0,-2.583892
3,0,3,;waitfor delay '0:0:__TIME__'--,;waitfor#~fx8;?B\ndelay#~fx8;?B\n'0:0:__TIME__'--,15,;waitfor#~fx8;?B\ndelay#~fx8;?B\n'0:0:__TIME__'--,-0.148936,0,-2.436445
4,0,4,;waitfor delay '0:0:__TIME__'--,;waitfor#~fx8;?B\ndelay#~fx8;?B\n'0:0:__TIME__'--,11,;waitfor#~fx8;?B\ndelay#~fx8;?B\n'0:0:__TIME__'--,-0.148936,0,-2.288998
5,0,5,;waitfor delay '0:0:__TIME__'--,;waitfor#~fx8;?B\ndelay#~fx8;?B\n'0:0:__TIME__'--,22,;waitfor#~fx8;?B\ndelay#~fx8;?B\n'0:0:__TIME__'--,-0.148936,0,-2.141552
6,0,6,;waitfor delay '0:0:__TIME__'--,;waitfor#~fx8;?B\ndelay#~fx8;?B\n'0:0:__TIME__'--,3,;waitfor#~fx8;?B\ndelay#~fx8;?B\n'0:0:__TIME__'--,-0.148936,0,-1.994105
7,0,7,;waitfor delay '0:0:__TIME__'--,;waitfor#~fx8;?B\ndelay#~fx8;?B\n'0:0:__TIME__'--,14,;waitfor#~fx8;?B\ndelay#~fx8;?B\n'0:0:__TIME__'--,-0.148936,0,-1.846658
8,0,8,;waitfor delay '0:0:__TIME__'--,;waitfor#~fx8;?B\ndelay#~fx8;?B\n'0:0:__TIME__'--,12,;waitfor#~fx8;?B\ndelay#~fx8;?B\n'0:0:__TIME__'--,-0.148936,0,-1.699211
9,0,9,;waitfor delay '0:0:__TIME__'--,;waitfor#~fx8;?B\ndelay#~fx8;?B\n'0:0:__TIME__'--,6,;waitf || #~fx8;?B\ndelay#~fx8;?B\n'0:0:__TIME...,-0.142857,0,-1.551764


In [11]:
# we have to access to the ppo policy in order to compute the prob(action|state)
# load ppo policy from checkpoint
tf.compat.v1.enable_eager_execution()
ray.init(local_mode='local-model')
algo = Algorithm.from_checkpoint("../src/ckpt_ppo_agent_tf2/checkpoint_000006")
ppo_policy = algo.get_policy()

/anaconda/envs/oppe4rl_22/lib/python3.9/site-packages/ray/_private/client_mode_hook.py:105: DeprecationWarning: DeprecationWarning: local mode is an experimental feature that is no longer maintained and will be removed in the future.For debugging consider using Ray debugger. 
  return func(*args, **kwargs)
2023-03-14 15:29:15,275	INFO worker.py:1538 -- Started a local Ray instance.
2023-03-14 15:29:16,229	INFO algorithm_config.py:2492 -- Executing eagerly (framework='tf2'), with eager_tracing=tf2. For production workloads, make sure to set eager_tracing=True  in order to match the speed of tf-static-graph (framework='tf'). For debugging purposes, `eager_tracing=False` is the best choice.
2023-03-14 15:29:16,245	INFO algorithm.py:501 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2023-03-14 15:29:16,304	WARNING worker.py:837 -- `ray.get_gpu_ids()` will always return the empty list when called from the driver. This is be

:actor_name:RolloutWorker


RayTaskError(AttributeError): [36mray::RolloutWorker.apply::Exiting()[39m (pid=8675, ip=10.0.0.5, repr=<ray.rllib.evaluation.rollout_worker._modify_class.<locals>.Class object at 0x7f5fe87bea30>)
  File "/anaconda/envs/oppe4rl_22/lib/python3.9/site-packages/ray/rllib/utils/actor_manager.py", line 183, in apply
    raise e
  File "/anaconda/envs/oppe4rl_22/lib/python3.9/site-packages/ray/rllib/utils/actor_manager.py", line 174, in apply
    return func(self, *args, **kwargs)
  File "/anaconda/envs/oppe4rl_22/lib/python3.9/site-packages/ray/rllib/evaluation/worker_set.py", line 607, in <lambda>
    lambda w: w.assert_healthy()
  File "/anaconda/envs/oppe4rl_22/lib/python3.9/site-packages/ray/rllib/evaluation/rollout_worker.py", line 857, in assert_healthy
    is_healthy = self.policy_map and self.input_reader and self.output_writer
AttributeError: 'RolloutWorker' object has no attribute 'policy_map'